# Glue Lab | Criando um Job

- O script a seguir tem sua execução final pela plataforma da AWS, para execução em um ambiente de desenvolvimento, crie um container docker com toda instalação necessária com este comando:

    `docker run -it -v ~/.aws:/home/glue_user/.aws -v $JUPYTER_WORKSPACE_LOCATION:/home/glue_user/workspace/jupyter_workspace/ -e AWS_PROFILE=default -e DISABLE_SSL=true -p 8888:8888 --name glue_jupyter_lab amazon/aws-glue-libs:glue_libs_4.0.0_image_01 /home/glue_user/jupyter/jupyter_start.sh`

- Você deverá ter configurado as permissões de acesso pela plataforma e pelo terminal do docker com o comando: `aws configure`.

## Links de acesso rápido

- [AWS Glue | Documentação](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-programming-python.html)

## Script Final

- O script abaixo realiza a leitura de um arquivo em `.csv` (`S3_INPUT_PATH`) e retorna no próprio s3 os dados organizados por partições, passando como parâmetros as colunas por ordem de hierarquia. 
- No final, após execução na plataforma da AWS, espera-se obter o seguinte resultado:

![particionamento](img/particionamento_glue.png)

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql.functions import upper
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_INPUT_PATH', 'S3_TARGET_PATH'])

# Criação dos objetos
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Início do Job
job.init(args['JOB_NAME'], args)


source_file = args['S3_INPUT_PATH']
target_path = args['S3_TARGET_PATH']

# Criação do dataframe
## Dynamic
nomes_dy = glueContext.create_dynamic_frame.from_options(
    
    "s3",
    {
        "paths": [source_file]
    },
    "csv",
    {
        "withHeader": True, "separator": ","
    }
    
    )

# Transformações
## 1. Passando o DY para DF Spark
nomes_df = nomes_dy.toDF()

# 2. Impressão do schema do DataFrame gerado
## Dynamic
# nomes_dy.printSchema()
## Spark
nomes_df.printSchema()


# 3. Colocar os valores da coluna 'nome' em Maiúsculo
## Dynamic
# maiuscula = lambda nome: nome.upper() if nome is not None else None
# nome_maiusc_col = Map.apply(frame=nomes, f = maiuscula)
## Spark
nome_maiusc_col = nomes_df.withColumn("nome", upper(nomes_df["nome"]))

# Outputs
## Dynamic
#glueContext.write_dynamic_frame.from_options(
#    
#    frame = nome_maiusc_col,
#    connection_type = "s3",
#    connection_options = {"path": target_path, "partitionKeys": ["sexo", "ano"]},
#    format = "json"
#    )
## Spark
nome_maiusc_col.write.partitionBy("sexo", "ano").mode('overwrite').json(target_path)

# Fim do Job
job.commit()


### Dicas

- Subir o script na plataforma e executar pelo container com os seguintes comandos:

- Run: `aws glue start-job-run --job-name SEU_NOME_DE_JOB`
- Log (json): `aws glue get-job-run --job-name MeuJobGlue --run-id SEU_ID_DE_EXECUCAO`

## Outras transformações 

In [ ]:
## Declarações
schema = "nome STRING, sexo STRING, total INT, ano INT"
nomes = spark.read.csv("/home/glue_user/workspace/jupyter_workspace/dados_brutos/nomes.csv", header=True, schema=schema)


## Transformações 
# 1. Colocar os valores da coluna 'nome' em Maiúsculo
# Resposta
from pyspark.sql.functions import upper
nome_maiusc_col = nomes.withColumn("nome", upper(nomes["nome"]))

# 2. Contagem das linhas do DataFrame
# Resposta
linhas = nomes.count()

# 3. Contagem de nomes agrupado por ano e sexo, e ordenado por ano de modo decrescente
# Resposta
from pyspark.sql.functions import count
contagem_nomes = nomes.groupBy("ano", "sexo").agg(count("nome").alias("contagem"))
contagem_nomes_ano = contagem_nomes.orderBy("ano", ascending=False)

# 4. Nome feminino mais registrado e o respectivo ano de registro
# Resposta
from pyspark.sql.functions import *
from pyspark.sql.window import Window

janela_analise = Window.partitionBy("ano").orderBy(desc("total"))

nomes_fem = nomes.filter(col('sexo') == "F")
nfs_mais_registrados = nomes_fem.withColumn("rank", dense_rank().over(janela_analise))
nf_mais_registrado_ano = nfs_mais_registrados.filter(nfs_mais_registrados.rank==1)
nf_mais_registrado_final = nf_mais_registrado_ano.select("nome", "ano", "total").orderBy("total", ascending=False)

# 5. Nome masculino mais registrado e o respectivo ano de registro

# Resposta
nomes_masc = nomes.filter(nomes.sexo == "M")
nms_mais_registrados = nomes_masc.withColumn("rank", dense_rank().over(janela_analise))
nm_mais_registrado_ano = nms_mais_registrados.filter(nms_mais_registrados.rank==1)
nm_mais_registrado_final = nm_mais_registrado_ano.select("nome", "ano", "total").orderBy("total", ascending=False)

# 6. Total de registros para cada ano (Apresentar as 10 primeiras linhas ordenada por Ano de modo crescente)

# Resposta
registros_ano = nomes.groupBy("ano").agg(sum("total").alias("total de registros por ano"))
registros_ordenados = registros_ano.orderBy("ano").limit(10)

## Modelo de Job | via AWS Glue


In [ ]:
import sys
from awsglue.context import GlueContext
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

JOB_NAME = 'Cola_Local'
S3_INPUT_PATH = 'dados_brutos/nomes.csv'
S3_TARGET_PATH = 'dados_processados/'

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_INPUT_PATH', 'S3_TARGET_PATH'])

# Criação dos objetos
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Início do Job
job.init(args['JOB_NAME'], args)


source_file = args['S3_INPUT_PATH']
target_path = args['S3_TARGET_PATH']

# Criação do dataframe
df = glueContext.create_dynamic_frame.from_options(
    
    "s3",
    {
        "paths": [source_file]
    },
    "csv",
    {
        "withHeader": True, "separator": ","
    }
    
)

# Transformações


# Outputs

glueContext.write_dynamic_frame.from_options(
    
    frame = only_1934,
    connection_type = "s3",
    connection_options = { "path":target_path },
    format = "parquet"

    )

# Fim do Job
job.commit()